In [ ]:
import ixmp
import yaml
import pyam
import nomenclature

In [ ]:
ar6_public = ixmp.Platform("ar6")
ar6_submission = ixmp.Platform("ar6-submission")

In [ ]:
ar6_public_scenario_list = ar6_public.scenario_list()
ar6_submission_scenario_list = ar6_submission.scenario_list()

In [ ]:
with open("region_rename.yaml") as f:
    region_rename_mapping = yaml.safe_load(f)

In [ ]:
eu_climate_dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
eu_climate = ixmp.Platform("eu-climate-submission")

In [ ]:
project = "LED"
scenario_selection = [("MESSAGEix-GLOBIOM 1.0", "LowEnergyDemand_1.3_IPCC")]

In [ ]:
project = "COVID-Recovery"
model = "MESSAGEix-GLOBIOM_1.2"
_scenarios = ar6_public_scenario_list[ar6_public_scenario_list.model == model].scenario

_scenarios = [s for s in _scenarios if s.startswith("COV_")]
_scenarios

In [ ]:
project = "GEI"
model = "MESSAGEix-GLOBIOM_GEI 1.0"
_scenarios = ar6_public_scenario_list[ar6_public_scenario_list.model == model].scenario

_scenarios = [s for s in _scenarios if s.startswith("SSP2_")]
_scenarios

In [ ]:
scenario_selection = list()
for s in _scenarios:
    scenario_selection.append((model, s))

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=ar6_submission, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.rename(region=region_rename_mapping, inplace=True)

In [ ]:
df.region

In [ ]:
df.rename(model={"MESSAGEix-GLOBIOM_1.2": "MESSAGEix-GLOBIOM 1.2"}, inplace=True)

In [ ]:
df.rename(model={"MESSAGEix-GLOBIOM_GEI 1.0": "MESSAGEix-GLOBIOM 1.0"}, inplace=True)

In [ ]:
gei_scenario_rename_mapping = dict([
    (i, i.replace("SSP2_", "GEI_SSP2_")) for i in df.scenario
])
df.rename(scenario=gei_scenario_rename_mapping, inplace=True)

In [ ]:
message_region_rename_mapping = dict([
    (i, i.replace("MESSAGEix-GLOBIOM_GEI 1.0", "MESSAGEix-GLOBIOM 1.0")) for i in df.region
])
df.rename(region=message_region_rename_mapping, inplace=True)

df.rename(region={"MESSAGEix-GLOBIOM 1.0|Sub-saharan Africa": "MESSAGEix-GLOBIOM 1.0|Sub-Saharan Africa"}, inplace=True)

In [ ]:
message_region_rename_mapping = dict([
    (i, i.replace("MESSAGEix-GLOBIOM_1.2", "MESSAGEix-GLOBIOM 1.2")) for i in df.region
])
df.rename(region=message_region_rename_mapping, inplace=True)

df.rename(region={"MESSAGEix-GLOBIOM 1.2|Sub-saharan Africa": "MESSAGEix-GLOBIOM 1.2|Sub-Saharan Africa"}, inplace=True)

In [ ]:
df.filter(variable=[
    "Expenditure Share|Food",
    "Disability-Adjusted Life Year|PM2.5",
    "GDP|PPP|Per capita growth rate",
    "GDP|PPP|Per capita ratio to OECD",
    "Premature Deaths|PM2.5",
], keep=False, inplace=True)

In [ ]:
df.rename(unit={
    "billion Int$2010/yr": "billion US$2010/yr",
    "bn vkm/yr": "billion vkm/yr",
    "bn pkm/yr": "billion pkm/yr",
    "bn tkm/yr": "billion tkm/yr",
    "bn m2/yr": "billion m2/yr",
    "Mt/year": "Mt/yr",
}, inplace=True)

In [ ]:
df.filter(region=["China", "India", "United States of America"], keep=False, inplace=True)

In [ ]:
df.filter(variable=eu_climate_dsd.variable, inplace=True)

In [ ]:
eu_climate_dsd.validate(df)

In [ ]:
project

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)
    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(eu_climate, _model, _scenario, version="new")
    scen.add_timeseries(df.data)
    scen.commit(f"Migrated from the IPCC AR6 database")
    scen.set_as_default()

    scen.set_meta({"Source": f"{project} (via IPCC AR6)"})
    
    print(f"Model {_model} - Scenario {_scenario}" )